# Imports
TODO: Add labels to dataloader

In [9]:
import os
import cv2
import time
import torch
import torchvision
import numpy as np
import pandas as pd
from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm2
import matplotlib.pyplot as plt

import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader


import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.nn.parameter import Parameter

from sklearn.model_selection import train_test_split

df = '../data/train.csv'
df = pd.read_csv(df)

In [10]:
train_df, val_df = train_test_split(df)

# Dataset

In [17]:
data = pd.read_feather(file)

In [19]:
data.index = data.image_id

In [20]:
data.head()

,image_id,0,1,2,3,4,5,6,7,8,...,16374,16375,16376,16377,16378,16379,16380,16381,16382,16383
image_id,,,,,,,,,,,,,,,,,,,,,
Train_0,Train_0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Train_1,Train_1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Train_2,Train_2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Train_3,Train_3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Train_4,Train_4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
labels = ['Train_100', 'Train_99', 'Train_1', 'Train_-1']
data.loc[data.index.intersection(set(labels))]

,image_id,0,1,2,3,4,5,6,7,8,...,16374,16375,16376,16377,16378,16379,16380,16381,16382,16383
Train_1,Train_1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Train_99,Train_99,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Train_100,Train_100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
df

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো
...,...,...,...,...,...
200835,Train_200835,22,7,2,র্খে
200836,Train_200836,65,9,0,ত্তো
200837,Train_200837,2,1,4,অ্যা
200838,Train_200838,152,9,0,স্নো


In [11]:
HEIGHT = 137
WIDTH = 236


def crop_resize(img0, size=(128, 128), pad=16):
    return torch.from_numpy(img[np.newaxis, ...].astype('float32'))

    
class BengaliAI(Dataset):
    def __init__(self, data, df, offset=0, imgsize=[128, 128]):
        
        self.data = data
        self.grapheme_roots = df['grapheme_root'].values
        self.vowel_diacritics = df['vowel_diacritic'].values
        self.consonant_diacritics = df['consonant_diacritic'].values
        self.offset = offset
        self._transforms = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Normalize([0.5], [0.5]),
                transforms.ToTensor(),
        ])
        self.h = 128


    def get_img(self, idx):
        img = self.data[idx, :].reshape(self.h, -1) #128x128
        return torch.from_numpy(img[np.newaxis, ...].astype('float32'))
#         return img[np.newaxis, ...] #1x128x128
    
    def __getitem__(self, idx):

        img = self.get_img(idx)
        idx += self.offset
        grapheme_root = self.grapheme_roots[idx]
        vowel_diacritic = self.vowel_diacritics[idx]
        consonant_diacritic = self.consonant_diacritics[idx]
        
        label = (grapheme_root, vowel_diacritic, consonant_diacritic)
        norm = transforms.Normalize([0.5], [0.5])
        img = norm(img)
        
#         img = self._transforms(img)        
        return img, label
        
    def __len__(self):
        return self.data.shape[0]


# Model

In [12]:
import pretrainedmodels

n_grapheme = 168
n_vowel = 11
n_consonant = 7
num_classes = [n_grapheme, n_vowel, n_consonant]
# out_dim = n_grapheme + n_vowel + n_consonant

class ClassifierCNN(nn.Module):
    def __init__(self, model_name, num_classes=num_classes):
        super(ClassifierCNN, self).__init__()
        
        self.inconv = nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=1, bias=True)
        self.model = pretrainedmodels.__dict__[model_name]()
        in_features = self.model.last_linear.in_features
        
        self.head_grapheme_root = nn.Linear(in_features, num_classes[0])
        self.head_vowel_diacritic = nn.Linear(in_features, num_classes[1])
        self.head_consonant_diacritic = nn.Linear(in_features, num_classes[2])
        
    def freeze(self):
        for param in self.model.parameters():
            param.requires_grad = False

    def unfreeze(self):
        for param in self.model.parameters():
            param.requires_grad = True
            
    def forward(self, x, logit=False):
        x  = self.inconv(x)
        features = self.model.features(x)
        features = F.adaptive_avg_pool2d(features, 1)
        features = features.view(features.size(0), -1)

        logit_grapheme_root = self.head_grapheme_root(features)
        logit_vowel_diacritic = self.head_vowel_diacritic(features)
        logit_consonant_diacritic = self.head_consonant_diacritic(features)
        
        if logit:
            return logit_grapheme_root, logit_vowel_diacritic, logit_consonant_diacritic            
        else:
            grapheme_root = F.softmax(logit_grapheme_root, 1)
            vowel_diacritic = F.softmax(logit_vowel_diacritic, 1)
            consonant_diacritic = F.softmax(logit_consonant_diacritic, 1)
            return grapheme_root, vowel_diacritic, consonant_diacritic            

In [13]:
# Dataloader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64
# directory = '../data/train_256/256/'
# train_dataset = BengaliAI(directory=directory)
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
device

device(type='cuda')

In [14]:
# Model, optim and scheduler
model_name = 'se_resnext101_32x4d'
# model_name = 'resnet34'
model = ClassifierCNN(model_name).to(device)
lr = 0.001
optimizer = optim.AdamW(
    model.parameters(), 
    lr=lr
)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10], gamma=0.3) 
criterion = nn.CrossEntropyLoss()

In [15]:
epochs = 5
model.freeze()
files = [f'../data/train_128_feather/train_{i}.feather' for i in range(4)]

batch_size = 64

if epochs:    
        
    for i in tqdm2(range(epochs)):
        running_loss = 0
        running_loss0 = 0
        running_loss1 = 0
        running_loss2 = 0
        offset = 0
        for file in files:
            data = pd.read_feather(file).iloc[:, 1:].values
            train_dataset = BengaliAI(data, df, offset=offset)
            train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                                                                  num_workers=8, shuffle=False)

            for i, (img, label) in enumerate(tqdm2(train_loader)):
                gStart = time.time()
                img = img.to(device)
                optimizer.zero_grad()

                start = time.time()
                out = model(img)    
                end = time.time()
                
#                 print(f"Forward pass time: {end-start}")
                
                start = time.time()
                loss0 = criterion(out[0], label[0].to(device))
                loss1 = criterion(out[1], label[1].to(device))
                loss2 = criterion(out[2], label[2].to(device))

                loss = loss0 + loss1 + loss2

                loss.backward()
                optimizer.step()
                end = time.time()
#                 print(f"Backward pass time: {end-start}")
                
                gEnd = time.time()
#                 print(f"Epoch time: {gEnd-gStart}")
            
            offset += len(train_dataset)    

        scheduler.step()
        print(f"Epoch: {i+1} | Loss: {running_loss} | [{running_loss0} | {running_loss1} | {running_loss2}]")

KeyboardInterrupt: 

In [ ]:
img

In [ ]:
files = [f'../data/train_128_feather/train_{i}.feather' for i in range(4)]
offset = 0
batch_size = 64
for file in files:
            data = pd.read_feather(file).iloc[:, 1:].values
            train_dataset = BengaliAI(data, df, offset=offset)
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

            for i, (img, label) in enumerate(tqdm2(train_loader)):
                continue
            offset += len(train_dataset)        

In [ ]:
files = [f'../data/train_128_feather/train_{i}.feather' for i in range(4)]
offset = 0
batch_size = 64
for file in files:
            data = pd.read_feather(file).iloc[:, 1:].values
            train_dataset = BengaliAI(data, df, offset=offset)
            train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=16, shuffle=False)

            for i, (img, label) in enumerate(tqdm2(train_loader)):
                continue
            offset += len(train_dataset)        